#  <font color='#660000'>Heart Failure Analysis

In [ ]:
from PIL import ImageTk, Image  
image1 = Image.open("../input/fgsgsgfrfxg/heart_the.png")
image1

#  <font color='#660000'>Contents

* [Indroduction to dataset](#2)
* [Importing Libraries and dataset](#3)
* [Data Preprocessing](#4)
* [Exploratory Data Analysis](#5)
* [Splitting and Scaling the data](#6)
* [Models](#7)
    * [KNN](#7.1)
    * [SVM](#7.2)
    * [Random Forest](#7.3)
    * [XG Boost](#7.4)
    * [Multilayer Perceptron](#7.5)
    * [compare the above models](#7.5)
* [Calculate AUC and plot ROC curve for two best models](#8)
* [Compare top 2 models](#9)

#  <font color='#660000'>Dataset <a id="2"></a>

### i) Source of dataset <a id="2"></a><a id="2.1"></a>

The dataset is taken from UCI Machine Learning Repository.The link to the dataset is given below:<br>
https://archive.ics.uci.edu/ml/datasets/Heart+Disease

### ii) Its Attributes <a id="2"></a><a id="2.2"></a>
  -   age -: age in years
  -   sex -: (1 = male; 0 = female)
  -   cp  -: chest pain type (4 values)
      - Value 0: typical angina
      - Value 1: atypical angina
      - Value 2: non-anginal pain
      - Value 3: asymptomatic
  -   trestbps -: resting blood pressure
  -   chol -:serum cholestoral in mg/dl
  -   fbs -: fasting blood sugar > 120 mg/dl (1 = true; 0 = false)
  -   restecg -: resting electrocardiographic results (values 0,1,2)
      - Value 0: normal
      - Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
      - Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
  -   thalach -: maximum heart rate achieved
  -   exang -: exercise induced angina (1 = yes; 0 = no)
  -   oldpeak -: ST depression induced by exercise relative to rest
  -   slope -: the slope of the peak exercise ST segment
      - Value 0: upsloping
      - Value 1: flat
      - Value 2: downsloping
  -   ca -: number of major vessels (0-3) colored by flourosopy
  -   thal -: 3 = normal; 6 = fixed defect; 7 = reversable defect
  -   target -: 0 = No heart disease, 1 = Heart disease is present

#  <font color='#660000'>Importing <a id="3"></a>

### i) Importing Libraries <a id="2"></a><a id="3.1"></a>

In [ ]:
import pandas as pd # Importing pandas for data manipulation and analysis
import numpy as np # Importing python linear algebra library to do work with arrays
from sklearn.model_selection import train_test_split # to split the data into train and test sets
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import *
from sklearn.preprocessing import MinMaxScaler#Importing the MinMaxSclaer from sklearn.preprocessing
from sklearn.preprocessing import StandardScaler#Importing the MinMaxSclaer from sklearn.preprocessing
from sklearn.model_selection import GridSearchCV
import warnings # to import warnings
warnings.filterwarnings('ignore') # to import warnings as 'ignore'
from sklearn.metrics import accuracy_score

### ii) Importing Dataset <a id="2"></a><a id="3.2"></a>

In [ ]:
df=pd.read_csv("../input/heart-disease-uci/heart.csv") # to import csv file as data frame

#  <font color='#660000'>Data Preprocessing <a id="4"></a>

### i) Reading the dataset <a id="4.1"></a>

In [ ]:
df

### ii) Checking missing values in the dataset <a id="4.2"></a>

In [ ]:
#Checking For Null values in our datasets and then removing the same.
pd.DataFrame(df.isna().sum()) #This will give the snapshot if me have any null values in our dataset.

### iii) Calculating percentage of missing values in the dataset <a id="4.3"></a>

In [ ]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True)
missing_value_df

### `Analysis of the output` -: As we can see from the above output that the dataset has no missing value, which is obviously a good thing.

#  <font color='#660000'>Exploratory Data Analysis <a id="5"></a>

### i) Datatype of each feature <a id="5.1"></a>

In [ ]:
df.info()

### ii) Statistical description of each feature <a id="5.2"></a>

In [ ]:
df.describe()

### iii) Univariant Analysis of Numerical feature <a id="5.2"></a>

In [ ]:
# visualizing numeric variables using seaborn
sns.set(font_scale=1.5)
sns.set_style(style='darkgrid')
f, axes = plt.subplots(2,3,figsize=(25,25))
sns.distplot( df["age"] ,hist_kws=dict(edgecolor="k", linewidth=2), color="#0000ff", ax=axes[0, 0])
sns.distplot( df["oldpeak"] ,hist_kws=dict(edgecolor="k", linewidth=2), color="#00cc00", ax=axes[0, 1])
sns.distplot( df["trestbps"] ,hist_kws=dict(edgecolor="k", linewidth=2), color="#e68a00", ax=axes[0, 2])
sns.distplot( df["chol"] , hist_kws=dict(edgecolor="k", linewidth=2),color="#992600", ax=axes[1, 0])
sns.distplot( df["thalach"] ,hist_kws=dict(edgecolor="k", linewidth=2), color="#e600ac", ax=axes[1, 1])

### `Analysis of the output` -: a) Age -: According to age histogram,the majority of the patients belonged to the age range between 50-65, and very less num of patients are under 35 years and even above 70 years.<br><br>b)Oldpeak -:As per the definition, the normal ST depression in ECG curve occurs during physical exercise and depression appears in the curve is generally less than 1mm, which will get fixed rapidly after the exercise is stopped. However,as per the histogram of old peak, we can see that the highest ST depression induced by exercise relative to rest shows highest frequency when it is below 1 mm, which means that most of the ST depression which appeared in patients ECG curve is normal,which get vanished as soon as patients stop doing physical exercise.<br><br>c) trestbps -: The highest value of resting blood pressure of patients who had been admitted to hospital is within the range of 120 mm Hg to 140 mm Hg. However, 120 mm Hg is considered as the normal blood pressure in humans.<br><br>d) chol -:The large number of patients are having cholestrol in between the range of 200 mg/dl to 300 mg/dl.We consider cholestrol level less than 200 mg/dl as normal. Therefore, according to the histogram, we can say most of the patients are having "borderline high" cholestrol level which raises the risk of heart disease.<br><br>e) thalach -: The maximum heart rate achieved by patients is in between 130 to 175. Generally,a normal resting heart rate of adults is within the range of 60 to 100 beats per minute.

### iv) Univariant Analysis of Categorical features <a id="5.3"></a>

In [ ]:
import matplotlib as mpl
mpl.rcParams['font.size'] =25
f, axes = plt.subplots(3,3,figsize=(29,29))
df['sex'].value_counts().plot.pie(autopct='%1.1f%%', pctdistance=0.5,colors=['red','green'], shadow=True, counterclock=True, startangle=90, wedgeprops={'linewidth':2, 'linestyle': 'solid', 'antialiased': True,'edgecolor':"k"},ax=axes[0, 0])
df['cp'].value_counts().plot.pie(autopct='%1.1f%%',pctdistance=0.6,colors=['blue','orange','red','olive'], shadow=True, counterclock=False, startangle=90, wedgeprops={'edgecolor':"k",'linewidth': 2, 'linestyle': 'solid', 'antialiased': True},ax=axes[0, 1])
df['fbs'].value_counts().plot.pie(autopct='%1.1f%%', pctdistance=0.6,colors=['blue','skyblue'], shadow=True, counterclock=False, startangle=90, wedgeprops={'edgecolor':"k",'linewidth': 2, 'linestyle': 'solid', 'antialiased': True},ax=axes[0, 2])
df['restecg'].value_counts().plot.bar(color=['olive','skyblue','red'],title='restecg',ax=axes[1, 0])
df['exang'].value_counts().plot.pie(autopct='%1.1f%%', pctdistance=0.6,colors=['olive','orange'], shadow=True, counterclock=False, startangle=90, wedgeprops={'edgecolor':"k",'linewidth': 2, 'linestyle': 'solid', 'antialiased': True},ax=axes[1, 1])
df['slope'].value_counts().plot.pie(autopct='%1.1f%%', pctdistance=0.6,colors=['olive','orange','red'], shadow=True, counterclock=False, startangle=90, wedgeprops={'edgecolor':"k",'linewidth': 2, 'linestyle': 'solid', 'antialiased': True},ax=axes[1, 2])
df['ca'].value_counts().plot.bar(color=['olive','orange','red','blue','skyblue'],title='ca',ax=axes[2, 0])
df['thal'].value_counts().plot.bar(color=['olive','orange','red','blue','skyblue'],title='thal',ax=axes[2, 1])
df['target'].value_counts().plot.pie(autopct='%1.1f%%', pctdistance=0.7,colors=['olive','orange','red','blue','skyblue'], shadow=True, counterclock=False, startangle=180, wedgeprops={'edgecolor':"k",'linewidth': 2, 'linestyle': 'solid', 'antialiased': True},ax=axes[2, 2])

### `Analysis of the output` -: a) sex -: According the pie chart, the proportion of male patients are nearly twice than female patients, which indicates that heart disease is more common in man than in woman.<br><br>b) cp -: The most common chest pain type in patients comes out to be typical angina which denoted by value 0.<br><br>c)fbs -:In most of the patients fasting blood sugar is less than 120 mg/dl which is depicted by the highest percentage of 0 value in pie chart. <br><br>d)restecg-:The highest number of patients have ST-T wave abnormality. However, according to the bar chart, still many of the patients have normal ecg results.<br><br>e)exang -: In most of the patients execise induced angina is not present.<br><br>f) slope -:In large number of patients, the slope of the peak exercise ST segment is either flat or downsloping.<br><br>g) ca -:Majority of patients have zero number of major vessels (0-3) colored by flourosopy.<br><br>h) thal-: Most of the heart disease patients are having thal stress value as 2, which indicates the fixed defect.<br><br>i) target -: There are higher chances, that patients who are admitted to hospital have heart disease.

### Multivariant Analysis of all the features with help of correlation matrix <a id="5.4"></a>

In [ ]:
corrMatrix = round(df.corr(),1)
plt.figure(figsize=(16,11))
sns.heatmap(corrMatrix,annot=True,cmap='seismic',linewidths=2,linecolor='black')
plt.title("Heatmap Correlation of Heart Failure Prediction", fontsize = 23)
plt.show()

### `Analysis of the output` -:- As per the heat map,the features that are extremely negatievly correlated are age & thalach,exang & cp, thalach & oldpeak, exang & thalach, exang & target, exang & cp, slop & oldpeak, oldpeak & target, ca & target.

#  <font color='#660000'>Splitting and Scaling <a id="6"></a>

### i) Splitting the training and testing sets <a id="6.1"></a>

In [ ]:
X = df.drop('target', axis=1).values #Feature datasets for the purpose of calculation.
y = df['target'].values #Target data sets for the purpose of calculations.
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, random_state=42)#Splitting the data into train and test sets.
pd.DataFrame(X_train) # to have a look at trained dataset

### ii) Scaling the data <a id="6.2"></a>

In [ ]:
#creating an object of Scaler
scaler = StandardScaler()

#Fitting the training features
scaler.fit(X_train)

#transforming the train features
X_train_scaled = scaler.transform(X_train)

#transforming the test features
X_test_scaled = scaler.transform(X_test)

#  <font color='#660000'>Models <a id="7"></a>

In [ ]:
train_accuracies=[]
test_accuracies=[]

### i) KNN Classifier <a id="7.1"></a>

In [ ]:
# Part 1 -: SELECT THE BEST HYPERPARMETERS WITH HELP OF GRID SEARCH

from sklearn.neighbors import KNeighborsClassifier
#List Hyperparameters that we want to tune.
Hyper_parameters = {'n_neighbors' : list(range(1,56)),'leaf_size' : list(range(1,50))}
# apply grid search to find best parameters for the model
GridSearch_knc = GridSearchCV(estimator = KNeighborsClassifier(), 
                           param_grid = Hyper_parameters,
                           cv = 10,
                           n_jobs = -1)
GridSearch_knc.fit(X_train_scaled, y_train)

print("Best hyperparameters for model:"+str(GridSearch_knc.best_params_))
print("Best estimator for model:"+str(GridSearch_knc.best_estimator_))

In [ ]:
#Part 2 -: Build a model with the help of best parameters
knc = KNeighborsClassifier(n_neighbors=16,leaf_size=1)
knc.fit(X_train_scaled, y_train)
# to predict the target values
pred_dt_test = knc.predict(X_test_scaled)
pred_dt_train = knc.predict(X_train_scaled)
train_accuracy_knn=accuracy_score(y_train,pred_dt_train)*100
test_accuracy_knn=accuracy_score(y_test,pred_dt_test)*100
train_accuracies.append(train_accuracy_knn)
test_accuracies.append(test_accuracy_knn)

# to find the accuracy of the model on training and testing data
print("Accuracy on  Train data : {}".format(accuracy_score(y_train,pred_dt_train)*100) )
print("Accuracy on  TEST data : {}".format(accuracy_score(y_test,pred_dt_test)*100) )

### ii) Support Vector Machine <a id="7.2"></a>

In [ ]:
# Part 1 -: SELECT THE BEST HYPERPARMETERS WITH HELP OF GRID SEARCH
from sklearn.svm import SVC 
# defining parameter range 
Hyper_parameters = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
GridSearch_svc = GridSearchCV(estimator = SVC(),
                               param_grid = Hyper_parameters,
                               cv = 15,
                               n_jobs = -1)
GridSearch_svc.fit(X_train_scaled, y_train)

print("Best hyperparameters for model:"+str(GridSearch_svc.best_params_))
print("Best estimator for model:"+str(GridSearch_svc.best_estimator_))

In [ ]:
#Part 2 -: Build a model with the help of best parameters
svc = SVC(C=10, gamma=0.01,kernel='rbf',probability=True)
svc.fit(X_train_scaled, y_train)
# to predict the target values
pred_svc_test = svc.predict(X_test_scaled)
pred_svc_train = svc.predict(X_train_scaled)
train_accuracy_svc=accuracy_score(y_train,pred_svc_train)*100
test_accuracy_svc=accuracy_score(y_test,pred_svc_test)*100
train_accuracies.append(train_accuracy_svc)
test_accuracies.append(test_accuracy_svc)
# to find the accuracy of the model on training and testing data
print("Accuracy on  Train data : {}".format(accuracy_score(y_train,pred_svc_train)*100) )
print("Accuracy on  TEST data : {}".format(accuracy_score(y_test,pred_svc_test)*100) )

### iii) Random Forest <a id="7.3"></a>

In [ ]:
# Part 1 -: Hypertuning of parameters
from sklearn.ensemble import RandomForestClassifier
#The structure that Scikit-learn needs to run Grid search
param_grid={'max_depth':[3,4,5],
           'max_leaf_nodes':[10,15,20],
            'min_samples_leaf':[10,15,20,25]}
from sklearn.model_selection import GridSearchCV
#applying GridSearch on a Decisiontree classifier with a 3 different parameters:
grid_search = GridSearchCV(RandomForestClassifier(n_estimators=11,random_state=573),param_grid,cv=10,return_train_score=True)
grid_search.fit(X_train_scaled,y_train)
print("Best parameters:"+str(grid_search.best_params_))
print("Best estimator:"+str(grid_search.best_estimator_))

In [ ]:
#Part 2 -: Build a model with the help of best parameters
rf = RandomForestClassifier(max_depth=5, max_leaf_nodes=15, min_samples_leaf=10,
                       n_estimators=11, random_state=573)
rf.fit(X_train_scaled, y_train)
# to predict the target values
pred_rf_test = rf.predict(X_test_scaled)
pred_rf_train = rf.predict(X_train_scaled)
train_accuracy_rf=accuracy_score(y_train,pred_rf_train)*100
test_accuracy_rf=accuracy_score(y_test,pred_rf_test)*100
train_accuracies.append(train_accuracy_rf)
test_accuracies.append(test_accuracy_rf)
# to find the accuracy of the model on training and testing data
print("Accuracy on  Train data : {}".format(accuracy_score(y_train,pred_rf_train)*100) )
print("Accuracy on  TEST data : {}".format(accuracy_score(y_test,pred_rf_test)*100) )

### iv) XG Boost <a id="7.4"></a>

In [ ]:
# Part 1 -: Hypertuning of parameters
#XGBoost
from xgboost import XGBClassifier
xg=XGBClassifier(random_state=573)

#List Hyperparameters that we want to tune.

parameter_grid_xg={'learning_rate':[0.05, 0.10, 0.15, 0.20],'max_depth':[3,4,5],'gamma':[ 0.0, 0.1, 0.2 , 0.3]}
gridsearch_xg = GridSearchCV(xg, parameter_grid_xg,cv=10)
gridsearch_xg.fit(X_train_scaled, y_train);

#Get best hyperparameters
gridsearch_xg.best_params_

In [ ]:
#Part 2 -: Build a model with the help of best parameters
xg =XGBClassifier(gamma=0.1,learning_rate=0.05,max_depth=3,random_state=573)
xg.fit(X_train_scaled, y_train)
# to predict the target values
pred_xg_test = xg.predict(X_test_scaled)
pred_xg_train = xg.predict(X_train_scaled)
train_accuracy_xg=accuracy_score(y_train,pred_xg_train)*100
test_accuracy_xg=accuracy_score(y_test,pred_xg_test)*100
train_accuracies.append(train_accuracy_xg)
test_accuracies.append(test_accuracy_xg)
# to find the accuracy of the model on training and testing data
print("Accuracy on  Train data : {}".format(accuracy_score(y_train,pred_xg_train)*100) )
print("Accuracy on  TEST data : {}".format(accuracy_score(y_test,pred_xg_test)*100) )

### v) Multilayer Perceptron <a id="7.5"></a>

In [ ]:
# Part 1 -: Hypertuning of parameters
from sklearn.neural_network import MLPClassifier

# defining parameter range 
Hyper_parameters = {'solver':['lbfgs'], 'alpha':[1e-1,1e-5,1e-10],
                    'hidden_layer_sizes':[(5, 2)], 'random_state':[1,6]}
GridSearch_mlp = GridSearchCV(estimator = MLPClassifier(),
                               param_grid = Hyper_parameters,cv=10,n_jobs=-1)
GridSearch_mlp.fit(X_train_scaled, y_train)

print("Best hyperparameters for model:"+str(GridSearch_mlp.best_params_))
print("Best estimator for model:"+str(GridSearch_mlp.best_estimator_))

In [ ]:
#Part 2 -: Build a model with the help of best parameters
mlp = MLPClassifier(alpha=0.1, hidden_layer_sizes=(5, 2), random_state=6,
              solver='lbfgs')
mlp.fit(X_train_scaled, y_train)
# to find the accuracy of the model on training and testing data
# to predict the target values
pred_mlp_test = mlp.predict(X_test_scaled)
pred_mlp_train = mlp.predict(X_train_scaled)
train_accuracy_mlp=accuracy_score(y_train,pred_mlp_train)*100
test_accuracy_mlp=accuracy_score(y_test,pred_mlp_test)*100
train_accuracies.append(train_accuracy_mlp)
test_accuracies.append(test_accuracy_mlp)
# to find the accuracy of the model on training and testing data
print("Accuracy on  Train data : {}".format(accuracy_score(y_train,pred_mlp_train)*100) )
print("Accuracy on  TEST data : {}".format(accuracy_score(y_test,pred_mlp_test)*100) )

### vi) Comparing the test and train accuracies of each model  <a id="7.6"></a>

In [ ]:
label = ['KNN','SVC','Random Forest','XG Boost','MLP']
print(label)

#checking the train and test accuracies for all the parameter values
train_accurac = [round(num, 2) for num in train_accuracies]
print("Train Accuracies "+str(train_accurac))

test_accurac = [round(num, 2) for num in test_accuracies]
print("\nTest Accuracies "+str(test_accurac))

In [ ]:
#Accuracy dataframe
Acc_df = pd.DataFrame({'Model':label,'Train Accuracy(%)': train_accurac,'Test Accuracy(%)': test_accurac})
Acc_df.style.background_gradient(cmap='Reds')

### `Analysis of the output` -:Here, I will consider 'MLP' and 'XGBoost' as my top 2 models as there train and test accuracies are having good percentage values. I am not taking random forest in top 2 models because it seems like random forest model is overfitting the data as its value of test accuracy is more than train accuracy, which means that this model is outperforming for unseen data.

#  <font color='#660000'>Calculate AUC values for selected model and Plot their ROC curve <a id="8"></a>

### i) XG Boost<a id="7.1"></a>

In [ ]:
# to plot roc curve for this model
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
# generate a no skill prediction (majority class)
ns_probs = [0 for _ in range(len(y_test))]
# Compute predicted probabilities: y_pred_prob
y_pred_prob = xg.predict_proba(X_test_scaled)[:,1]
# Compute and print AUC score
auc_xg=roc_auc_score(y_test, y_pred_prob)*100
print(" ----AUC:----- {}".format(roc_auc_score(y_test, y_pred_prob)*100))
#Plot ROC Curve
fpr_dt, tpr_dt, _ = roc_curve(y_test,y_pred_prob)# to plot random chances
ns_fpr,ns_tpr, _ = roc_curve(y_test, ns_probs)# to plot roc curve for the model
# plot the roc curve for the model
plt.plot(ns_fpr, ns_tpr, linestyle='--', label='Random Chances',color='Blue') # plot the random chances for the model
plt.plot(fpr_dt, tpr_dt, marker='.', label='ROC Curve',color='Red')# plot the roc curve for the model
# axis labels
plt.xlabel('False Positive Rate',fontsize=20)
plt.ylabel('True Positive Rate',fontsize=20)
plt.title('Receiver Operating Characteristics',fontsize=20,color='green')
plt.legend()
# show the legend
# show the grid
plt.grid()
# show the plot
print('**************************************************************************************************')

### ii) MLP <a id="7.2"></a>

In [ ]:
# to plot roc curve for this model
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
# generate a no skill prediction (majority class)
ns_probs = [0 for _ in range(len(y_test))]
# Compute predicted probabilities: y_pred_prob
y_pred_prob = mlp.predict_proba(X_test_scaled)[:,1]
# Compute and print AUC score
auc_mlp=roc_auc_score(y_test, y_pred_prob)*100
print(" ----AUC:----- {}".format(roc_auc_score(y_test, y_pred_prob)))
#Plot ROC Curve
fpr_dt, tpr_dt, _ = roc_curve(y_test,y_pred_prob)# to plot random chances
ns_fpr,ns_tpr, _ = roc_curve(y_test, ns_probs)# to plot roc curve for the model
# plot the roc curve for the model
plt.plot(ns_fpr, ns_tpr, linestyle='--', label='Random Chances',color='Blue') # plot the random chances for the model
plt.plot(fpr_dt, tpr_dt, marker='.', label='ROC Curve',color='Red')# plot the roc curve for the model
# axis labels
plt.xlabel('False Positive Rate',fontsize=20)
plt.ylabel('True Positive Rate',fontsize=20)
plt.title('Receiver Operating Characteristics',fontsize=20,color='green')
plt.legend()
# show the legend
# show the grid
plt.grid()
# show the plot
print('**************************************************************************************************')

#  <font color='#660000'>Compare MLP and XG Boost <a id="9"></a>

In [ ]:
label = ['XG Boost','MLP']
train_acc = [round(train_accuracy_xg,2),round(train_accuracy_mlp,2)]

test_acc = [round(test_accuracy_xg,2),round(test_accuracy_mlp,2)]
auc_values=[round(auc_xg,2),round(auc_mlp,2)]

In [ ]:
x = np.arange(len(label))  
width = 0.1
# Figure Size 
fig, ax = plt.subplots(figsize=(15, 11))

# creating the bar plot
bar1 = ax.bar(x - width/2, train_acc, width, label='Train',color='olive')
bar2 = ax.bar(x + width/5, test_acc, width, label='Test',color='orange')
bar3 = ax.bar(x + width, auc_values, width, label='AUC Score',color='red')


# put labels
ax.set_ylabel('Score',fontsize=10,fontweight='bold')
ax.set_xlabel('Models',fontsize=10,fontweight='bold')
ax.set_title('Comparison of models',fontsize=21,fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(label)
ax.set_ylim([0, 110])
ax.legend(loc='center right', bbox_to_anchor=(1.19, 0.9))

# to annotate
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",fontsize=10,fontweight='bold')
autolabel(bar1)
autolabel(bar2)
autolabel(bar3)
plt.grid(b = True, color ='black', linestyle ='-', linewidth = 0.7, alpha = 1.0)

plt.show()

### `Analysis of the output` -:According to the output it seems like MLP is the best model as its train and test accuracies are way more better than XG boost. Moreover, its AUC value is 85.25% which means MLP model is 85% capable of predicting 1 as 1 and 0 as 0.<br> However, the dataset is small in size, therefore, the model might not met with today's industrial standards.